In [5]:
import pymysql

In [6]:
envs = dict([l.strip().split('=') for l in open('.env', 'r').readlines()])

In [7]:
db_config = {}
for k, v in envs.items():
    if 'DB' not in k:
        continue
    k = k.split('_')[1].lower()
    
    if k == 'port':
        v = int(v)

    db_config[k]=v
conn = pymysql.connect(**db_config)

In [ ]:
# tours 테이블 생성
with conn.cursor() as cur:
    cur.execute("""
    CREATE TABLE tours (
    tour_id int not null,
    title varchar(1024),
    country varchar(255),
    tour_days int,
    price int unsigned,
    tour_url varchar(1024), 
    start_location varchar(255),
    end_location varchar(255),
    style varchar(255),
    type varchar(255),
    operator varchar(255),
    primary key(tour_id)
    );       
    """)
conn.commit()

In [ ]:
# locations 테이블 생성
with conn.cursor() as cur:
    cur.execute("""
    CREATE TABLE locations (
    location_id int not null auto_increment,
    tour_id int not null,
    location_name varchar(1024),
    stay_days int,
    location_url varchar(1024),
    latitude double,
    longitude double,               
    primary key(location_id),
    foreign key (tour_id) references tours(tour_id)
    );       
    """)
conn.commit()

In [ ]:
# location_big_five 테이블 생성
with conn.cursor() as cur:
    cur.execute("""
    CREATE TABLE location_big_five (
    id int not null auto_increment,
    location_id int not null,
    big_five_animal varchar(1024),
    primary key(id),
    foreign key (location_id) references locations(location_id)
    );       
    """)
conn.commit()

In [ ]:
# tour_activities 테이블 생성
with conn.cursor() as cur:
    cur.execute("""
    CREATE TABLE tour_activities (
    id int not null auto_increment,
    tour_id int not null,
    activity varchar(1024),
    primary key(id),
    foreign key (tour_id) references tours(tour_id)
    );       
    """)
conn.commit()

In [8]:
import json

In [9]:
with open('Data/Safari_tour_add_lat_and_lon.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [20]:
len(data)

13429

In [7]:
tours_data = []
for tour in data:
    tours_data.append((
        int(tour['Tour_ID']),
        tour['Tour_title'],
        tour['Tour_country'],
        int(tour['Tour_days']),
        int(tour['Tour_price']),
        tour['Tour_URL'],
        tour['Tour_start'],
        tour['Tour_End'],
        tour['Tour_style'],
        tour['Tour_type'],
        tour['Tour_operator']
    ))

In [8]:
with conn.cursor() as cur:
    cur.executemany("""
    insert into tours (tour_id, title, country, tour_days, price, tour_url, start_location, end_location, style, type, operator)
    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", tours_data)
conn.commit()

In [27]:
locations_data = []
for tour in data:
    tour_id = int(tour['Tour_ID'])
    
    for loca in tour['Tour_location_details']:
        if 'location_name' in loca:
            location_name = loca['location_name']
        else:
            location_name = ''
        if 'days' in loca:
            stay_days = loca['days']
        else:
            stay_days = None
        if 'link' in loca:
            location_url = loca['link']
        else:
            location_url = ''
        if 'lat' in loca and loca['lat'] != '':
            latitude = loca['lat']
        else:
            latitude = None
        if 'lon' in loca and loca['lon'] != '':
            longitude = loca['lon']
        else:
            longitude = None

        locations_data.append((
        tour_id,
        location_name,
        stay_days,
        location_url,
        latitude,
        longitude
        ))


In [28]:
with conn.cursor() as cur:
    cur.executemany("""
    insert into locations (tour_id, location_name, stay_days, location_url, latitude, longitude)
    values (%s, %s, %s, %s, %s, %s)
    """, locations_data)
conn.commit()

In [1]:
# location_big_five 데이터 insert

In [19]:
with open('Data/Safari_tour_add_lat_and_lon.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [20]:
location_big_five = []
with conn.cursor(pymysql.cursors.DictCursor) as cur:
    result = cur.execute('select location_id, tour_id, location_name from locations')
    locations_data = cur.fetchall()

for location_table in locations_data:
    location_id = location_table['location_id']
    tour_id = location_table['tour_id']
    location_name = location_table['location_name']

    for tour in data:
        if int(tour['Tour_ID']) == tour_id:
            for loca in tour['Tour_location_details']:
                if 'location_name' in loca and loca['location_name'] == location_name:
                    if 'animals' in loca and loca['animals']:
                        for animals in loca['animals']:
                            location_big_five.append((location_id, animals))
            break

In [22]:
with conn.cursor() as cur:
    cur.executemany("""
    insert into location_big_five (location_id, big_five_animal)
    values (%s, %s)
    """, location_big_five)
conn.commit()

In [2]:
# tour_activities 데이터 insert

In [23]:
tour_activities = []
for tour in data:
    tour_id = int(tour['Tour_ID'])
    if 'Tour_activity' in tour and tour['Tour_activity']:
        for activity in tour['Tour_activity']:
            tour_activities.append((tour_id, activity))

In [24]:
with conn.cursor() as cur:
    cur.executemany("""
    insert into tour_activities (tour_id, activity)
    values (%s, %s)                    
    """, tour_activities)
conn.commit()

In [17]:
conn.close()